In [1]:
import re
from pathlib import Path
import os
import time


Variation ...
* N1_kulturberikaren  --  behövs kanske inte
* N1_kulturberikare (lemma)
* N1C_globalistmaffia -- behövs nog inte heller
* N1C_globalistX
* N1C_globalist maffia 


##### Original
0. sv_compounds (original-original)
1. en_compounds (original-enc)
##### Lemmatized
2. sv_compounds (lem-original)
3. en_compounds (len-enc)
    

In [2]:
corpus_in = Path("/home/max/Corpora/fb-pol-corpus/fb-pt-clean/yearly")
#corpus_out = Path("/home/max/Corpora/fb-pol-corpus/fb-pt-radical1/yearly")

In [3]:
paradigm = Path("utils/dwts.paradigm")

In [4]:
def read_util(file_path):
    with open(file_path, "r") as f:
        terms = [line.strip("\n").split("#")[0] for line in f.readlines()]
    terms = [term for term in terms if term != ""]
    return terms 

In [5]:
def read_paradigm(file_path):
    paradigms = read_util(file_path)
    paradigms = [tuple([re.sub(r" \Z", r"", column) for column in p.split(" -> ")]) for p in paradigms]
    return paradigms

In [6]:
def enc(line, paradigm):
   
    for pos, regex, lemma in paradigm:
        if lemma.endswith("X"):
            regex = re.compile(regex)
            if re.search(regex, line):
                #print(1, lemma, i, line)

                line = re.sub(regex, r"\1 \2", line)

                #print(2, lemma, i, line)
        
    return line
    
def dwt_lem(line, paradigm):
    
    for pos, regex, lemma in paradigm:
        if lemma == "hjälpa_X_på_plats":
            continue
        regex = re.compile(regex)
        line = re.sub(regex, f"{pos}_{lemma}", line)
    
    return line

In [7]:
def dwt_transformer(line, paradigm, mode):
        
    if mode == 1:
        return enc(line, paradigm)
    if mode == 2:
        return dwt_lem(line, paradigm)
    if mode == 3:
        line = enc(line, paradigm)
        line = dwt_lem(line, paradigm)
        return line

In [8]:
def preprocess_dwt(corpus_in, corpus_out, full_paradigm, mode=1):
    
    #global i
    
    t0 = time.time()
    corpus_in = Path(corpus_in)
    corpus_out = Path(corpus_out)
    files = sorted(os.listdir(corpus_in))
    
    roots, pos, regex, lemmas = zip(*read_paradigm(full_paradigm))

    paradigm = list(zip(pos, regex, lemmas))
    roots = set(roots)
    roots = re.compile(r"(" + "|".join(roots) + ")")

    for k, file in enumerate(files, start=1):
        
        f_out = open(corpus_out / file, "w")
        f_out.close()
        
        with open(corpus_in / file, "r") as f_in, open(corpus_out / file, "a") as f_out:
            year = file.strip(".txt")
            
            for i, line in enumerate(f_in):
                
#                 if k > 3:
#                     return 
                
                if i % 10000 == 0:
                    print(f"{file} {k} / {len(files)}: {i}    ", end="\r")
                
                if re.search(roots, line) == None:
                    f_out.write(line)
                    continue                
                
                line = dwt_transformer(line, paradigm, mode)
                
                #print(line)
                
                f_out.write(line)

    
    delta_t = time.time() - t0
    m = int(delta_t / 60)
    s = int(delta_t / 60)
    
    print()
    print("Done!", f"({m} m, {s} s)")             

In [9]:
def template(x):
    return f"/home/max/Corpora/fb-pol-corpus/fb-pt-radical{x}/yearly"

for mode in [1, 2, 3]:
    print()
    print("MODE =", mode)
    print()
    preprocess_dwt(corpus_in, template(mode), paradigm, mode=mode)


MODE = 1

2022.txt 23 / 23: 3610000    
Done! (3 m, 3 s)

MODE = 2

2022.txt 23 / 23: 3610000    
Done! (3 m, 3 s)

MODE = 3

2022.txt 23 / 23: 3610000    
Done! (3 m, 3 s)
